# Getting started

This tutorial shows you how to get started with the `povm_toolbox`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from povm_toolbox.library.pm_sim_implementation import ClassicalShadows, RandomizedPMs
from povm_toolbox.post_processor.povm_post_processor import POVMPostprocessor
from povm_toolbox.sampler.povm_sampler import POVMSampler
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import (
    Operator,
    SparsePauliOp,
    Statevector,
    random_hermitian,
)
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import Session

## Random Circuit

We first llok at a random the 2-qubit circuit, with depth 3.

In [3]:
n_qubit = 3
n_cicuits = 5
np.random.seed(10)
seeds= np.random.randint(0,10000000,size=n_cicuits)
qc_random = [random_circuit(num_qubits=n_qubit, depth = 3, measure=False, seed=seed) for seed in seeds]
qc_random[0].draw()

┌──────┐┌──────┐                                                
q_0: ─────┤1     ├┤0     ├──────────────────────────────■─────────────────
     ┌───┐│      ││      │    ┌───┐    ┌────────────────┴────────────────┐
q_1: ┤ H ├┤  Dcx ├┤  Ecr ├────┤ S ├────┤ U(3.4168,2.3054,0.68636,1.6934) ├
     └───┘│      ││      │┌───┴───┴───┐└─────────────────────────────────┘
q_2: ─────┤0     ├┤1     ├┤ P(5.0287) ├───────────────────────────────────
          └──────┘└──────┘└───────────┘

We also draw some random observables 

In [4]:
set_observables = [SparsePauliOp.from_operator(random_hermitian(2**n_qubit)) for _ in range(10)]

For reference, we compute the true final state and the exact expectation values for the different observables.

In [5]:
exp_val = np.zeros((len(qc_random), len(set_observables)), dtype=complex)
for i, qc in enumerate(qc_random):
    exact_state = Statevector(np.array(Operator(qc).data[:,0]).squeeze())
    exp_val[i] = np.array([exact_state.expectation_value(obs) for obs in set_observables])
exp_val = np.real_if_close(exp_val)

## Classical Shadows

We now look at the implementation of Classical Shaodws measurement

In [6]:
# By default, the Classical Shadows (CS) measurement uses X,Y,Z measurements with equal probability.
cs_implementation = ClassicalShadows(n_qubit=n_qubit)
# Define the default shot budget.
cs_shots = 4096

In [7]:
pubs = []
for qcr in qc_random:
    pubs.append((qcr, None, np.random.randint(1000,10000) if np.random.randint(2)==0 else None, cs_implementation))
print(pubs)

[(<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x120c886a0>, None, None, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b71a80>, None, 5829, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b72290>, None, 7400, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b723e0>, None, 6648, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>), (<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x144b718a0>, None, 1239, <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>)]


In [8]:
# Run the sampler job locally using AerSimulator.
# Session syntax is supported but ignored because local mode doesn't support sessions.
backend = AerSimulator()
with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    cs_job = sampler.run(pubs, shots=cs_shots)

/Users/dao/Documents/Internship/ic-povms/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/session.py:156: UserWarning: Session is not supported in local testing mode or when using a simulator.
  warnings.warn(


### Results
We retrieve the result object, which contains the POVM used and from which we can query the counts of each outcome.

In [9]:
cs_result = cs_job.result()
print(f"POVM: {cs_result[0].povm_metadata}")
print(f"Counts: {cs_result[1].get_counts()}")

POVM: <povm_toolbox.library.pm_sim_implementation.ClassicalShadows object at 0x144b715d0>
Counts: Counter({(0, 4, 0): 231, (0, 1, 0): 132, (3, 4, 0): 119, (0, 4, 4): 118, (0, 4, 3): 116, (0, 4, 5): 114, (0, 3, 0): 109, (0, 4, 2): 106, (0, 2, 0): 105, (4, 4, 0): 99, (2, 4, 0): 97, (5, 4, 0): 94, (0, 0, 0): 87, (5, 4, 2): 71, (4, 0, 0): 68, (3, 4, 3): 65, (0, 3, 2): 63, (5, 0, 0): 63, (4, 4, 5): 62, (5, 2, 0): 61, (4, 1, 0): 60, (5, 1, 0): 60, (5, 4, 4): 59, (0, 1, 3): 59, (5, 4, 3): 59, (0, 3, 4): 59, (0, 0, 4): 58, (0, 0, 2): 58, (0, 1, 2): 57, (0, 1, 4): 57, (0, 2, 3): 57, (3, 4, 2): 57, (0, 0, 5): 56, (4, 4, 3): 56, (4, 3, 0): 54, (0, 3, 5): 53, (2, 0, 0): 53, (2, 3, 0): 53, (3, 4, 5): 53, (2, 4, 4): 53, (3, 4, 4): 53, (3, 2, 0): 52, (3, 1, 0): 52, (0, 2, 5): 52, (3, 3, 0): 50, (0, 0, 3): 50, (2, 1, 0): 50, (2, 4, 5): 49, (0, 1, 5): 49, (4, 2, 0): 49, (0, 2, 2): 48, (4, 4, 4): 48, (2, 4, 2): 48, (2, 4, 3): 48, (5, 4, 5): 46, (3, 0, 0): 46, (5, 3, 0): 45, (2, 2, 0): 44, (4, 4, 2): 43,

We now define our POVM post-processor, which will use the resukt object to estimate expectation values of some observables.

In [10]:
for i_state in range(len(cs_result)):
    cs_postprocessor = POVMPostprocessor(cs_result[i_state])
    cs_est_exp_val = [cs_postprocessor.get_expectation_value(obs) for obs in set_observables]
    cs_shots = sum(cs_postprocessor.counts.values())

    n = int(np.ceil(np.log10(cs_shots)))
    print('Exact        CS')
    print(f'             ({cs_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {cs_est_exp_val[i]:>{8+n}.3e}   {abs(cs_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        CS
             (4096 shots)
-1.925e-01     -3.091e-01      60.5%
 7.483e-02      8.750e-02      16.9%
-5.070e-01     -7.032e-01      38.7%
-1.542e+00     -1.566e+00       1.5%
 6.973e-01      7.886e-01      13.1%
-4.797e-01     -4.501e-01       6.2%
 8.208e-01      1.040e+00      26.7%
 2.912e-01      3.166e-01       8.7%
-3.418e-01     -3.272e-01       4.3%
-6.683e-01     -6.495e-01       2.8%
Exact        CS
             (5829 shots)
 2.214e-01      9.509e-02      57.1%
-6.653e-02     -3.361e-02      49.5%
 1.927e-01      1.623e-01      15.8%
-5.083e-01     -5.619e-01      10.5%
 1.283e+00      1.334e+00       4.0%
-3.314e-01     -4.154e-01      25.3%
 1.639e-01      2.225e-01      35.8%
-7.509e-01     -7.337e-01       2.3%
 1.907e-01      2.039e-01       6.9%
 1.645e+00      1.809e+00      10.0%
Exact        CS
             (7400 shots)
 4.679e-01      2.443e-01      47.8%
-3.805e-02     -1.470e-01     286.3%
-1.146e+00     -1.091e+00       4.8%
-4.356e-01     -5.579e-

## PM-simulable POVM
We now look at POVM that are simulable (through randomization) with only single-qubit projective measurements (PMs).

In [11]:
pubs_pm = []
for pub in pubs:
    pubs_pm.append((pub[0], None, pub[2], None))

# Define our projective measurements acting on each qubit.
angles = np.array([[0.0, 0.0, 0.5 * np.pi, 0.0, 0.5 * np.pi, 0.5 * np.pi],[1.2,0.0, 0.1,3.14,0.7,0.3], [0.2,0.1, 1.2,3.14,-0.7,0.3]])
# Set the distributions of the shots among the PMs.
bias = np.array([[0.34, 0.33, 0.33], [0.7,0.15,0.15], [0.34,0.33,0.33]])

# Define the PM-simulable POVM.
pmsim_implementation = RandomizedPMs(n_qubit=n_qubit, bias=bias, angles=angles)

In [12]:
pmsim_shots = 4096

with Session(backend=backend) as session:
    # First define a standard sampler (that will be used under the hood).
    runtime_sampler = Sampler(session=session)
    # Then define the POVM sampler, which takes BaseSampler as an argument.
    sampler = POVMSampler(runtime_sampler)
    # Submit the job by specifying which POVM to use, which circuit(s) to measure and the shot budget.
    pmsim_job = sampler.run(pubs_pm, povm=pmsim_implementation, shots=pmsim_shots)

### Results

Retrieve the result of the sampling and use it to estiamte some expectation values via the post-processor

In [13]:
pmsim_result = pmsim_job.result()

for i_state in range(len(pmsim_result)):
    pmsim_postprocessor = POVMPostprocessor(pmsim_result[i_state])
    pmsim_est_exp_val = [pmsim_postprocessor.get_expectation_value(obs) for obs in set_observables]
    pmsim_shots = sum(pmsim_postprocessor.counts.values())

    n = int(np.ceil(np.log10(pmsim_shots)))
    print('Exact        PM-sim')
    print(f'             ({pmsim_shots} shots)')
    for i in range(len(set_observables)):
        print(f'{np.real(exp_val[i_state,i]):>10.3e}   {pmsim_est_exp_val[i]:>{8+n}.3e}   {abs(pmsim_est_exp_val[i]-np.real(exp_val[i_state,i]))/abs(np.real(exp_val[i_state,i])):>{12-n}.1%}')

Exact        PM-sim
             (4096 shots)
-1.925e-01     -6.184e-01     221.2%
 7.483e-02      5.863e-01     683.5%
-5.070e-01     -2.441e+00     381.5%
-1.542e+00     -1.769e+00      14.7%
 6.973e-01      1.436e+00     106.0%
-4.797e-01     -2.509e+00     423.1%
 8.208e-01     -1.872e+00     328.1%
 2.912e-01      4.774e+00    1539.7%
-3.418e-01      1.277e+00     473.6%
-6.683e-01     -3.531e+00     428.4%
Exact        PM-sim
             (5829 shots)
 2.214e-01     -7.870e-01     455.5%
-6.653e-02      9.488e-01    1526.1%
 1.927e-01     -2.806e-01     245.6%
-5.083e-01     -3.178e-01      37.5%
 1.283e+00      1.353e+00       5.4%
-3.314e-01      4.258e-01     228.5%
 1.639e-01      1.334e-01      18.6%
-7.509e-01     -2.204e+00     193.5%
 1.907e-01      1.436e+00     652.8%
 1.645e+00      1.488e+00       9.6%
Exact        PM-sim
             (7400 shots)
 4.679e-01     -9.546e-01     304.0%
-3.805e-02      7.860e-01    2165.9%
-1.146e+00     -4.308e+00     276.0%
-4.356e-01 